<a href="https://colab.research.google.com/github/saciva-team-2b/Saciva-University-Recommendation-System-2B/blob/main/Saciva_University_Recommendation_System_2B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Saciva University Recommendation System 2B**



---


**Resources and Documents:**  
[Trello Board](https://trello.com/b/nXdTl8OF/saciva-university-recommendation-system-2b) | [Google Drive](https://drive.google.com/drive/folders/1Psq7fGf3CNVNYAeFybnAVz_SMj8Py3TY) | [Github Repo](https://github.com/saciva-team-2b/Saciva-University-Recommendation-System-2B)

---

### **Challenge Summary**
Suggest top 5 Higher Education universities to a student by matching student's with the universities' profile. Student Profile: Current Education level, Field of study, Chosen research, entrepreneurship, and financial status. University Profile: location (climate, cost of living), the field, overall ranking, fees and tuition, other factors.

### **Context / Impact**
Saciva is an app for Students, especially international students to help through their higher education journey and choosing a university is the first step and often research on academics and tuition and over look factors such as climate, opportunities after in their field, cost of living, safety, etc. before they take admission.

### **Suggested Approach**
The student's profile can be vectorized to find the nearest data (unsupervised learning) to suggest the 5 most suitable universities to the student.

In [23]:
from google.colab import userdata

!git config --global user.email "sacivateamb@gmail.com"
!git config --global user.name "saciva-team-2b"
!git config --global user.password {userdata.get('githubpass')}

token = {userdata.get('githubtoken')}
username = "saciva-team-2b"
repo = "Saciva-University-Recommendation-System-2B"

In [24]:
!git clone https://{token}@github.com/{username}/{repo}

Cloning into 'Saciva-University-Recommendation-System-2B'...
remote: Enumerating objects: 15, done.
remote: Counting objects: 100% (15/15), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 15 (delta 5), reused 6 (delta 1), pack-reused 0 (from 0)
Receiving objects: 100% (15/15), 4.38 KiB | 4.38 MiB/s, done.
Resolving deltas: 100% (5/5), done.


## **Data Exploration and Analysis**



First, we shall import all the required datasets. This includes:  
1. a
2. b
3. c

In [ ]:
# code starts here

Next, we shall import some standard ML libraries, and import more as we need.



---


### GitHub Commands


In [ ]:
%ls
!git add --all

 LICENSE   README.md  'Saciva University Recommendation System 2B.ipynb'


In [ ]:
!git status

On branch main
Your branch is up to date with 'origin/main'.

nothing to commit, working tree clean


In [ ]:
!git commit -a -m "Setup Data Exploration and Analysis"
!git push origin master

On branch main
Your branch is up to date with 'origin/main'.

nothing to commit, working tree clean
error: src refspec master does not match any
error: failed to push some refs to 'https://github.com/saciva-team-2b/Saciva-University-Recommendation-System-2B'
